In [1]:
# 导入模块
%matplotlib inline
%load_ext autoreload
%autoreload
import unicodecsv as csv
import codecs
import numpy as np
import torch
from torch import nn
import pylab as pl
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
# 导入数据
'''
max_sentence_length = 10

data_size = 200

# indexing the words
source_sentences = []
w2i_s = {'':0}
i2w_s = {0:''}

target_sentences = []                
w2i_t = {'':0}
i2w_t = {0:''}

n1 = 1
n2 = 1
with codecs.open('data/chat.txt','r') as f:
    data = csv.reader(f,delimiter='\t')
    for row in data:
        s1 = row[0].split(' ')[:max_sentence_length]
        s2 = row[1].split(' ')[:max_sentence_length]
        source_sentences.append(s1)
        target_sentences.append(s2)

        for w in s1:
            if not w in w2i_s: 
                w2i_s[w] = n1
                i2w_s[n1] = w
                n1 += 1
        for w in s2:
            if not w in w2i_t: 
                w2i_t[w] = n2
                i2w_t[n2] = w
                n2 += 1
        if len(target_sentences)>data_size:
            break

n_words_source = len(w2i_s)
n_words_target = len(w2i_t)
n_words = n_words_source + n_words_target
print(len(target_sentences), n_words)
'''

(201, 717)


In [3]:
# 导入数据
max_sentence_length = 30

#data_size = 200

# indexing the words
source_sentences = []
w2i_s = {'':0}
i2w_s = {0:''}

target_sentences = []                
w2i_t = {'':0}
i2w_t = {0:''}

n1 = 1
with codecs.open('data/training-es.txt','r') as f:
    data = csv.reader(f,delimiter=' ')
    for row in data:
        s1 = row[:max_sentence_length]
        source_sentences.append(s1)
        for w in s1:
            if not w in w2i_s: 
                w2i_s[w] = n1
                i2w_s[n1] = w
                n1 += 1

n2 = 1
with codecs.open('data/training-en.txt','r') as f:
    data = csv.reader(f,delimiter=' ')
    for row in data:
        s2 = row[:max_sentence_length]
        target_sentences.append(s2)
        for w in s2:
            if not w in w2i_t: 
                w2i_t[w] = n2
                i2w_t[n2] = w
                n2 += 1
            
data_size = len(target_sentences)
n_words_source = len(w2i_s)
n_words_target = len(w2i_t)
n_words = n_words_source + n_words_target
print(data_size, n_words)

(9900, 1201)


In [ ]:
# embedding using skip gram, i.e., the nearest
word_embedding_dim = 15
sentence_embedding_dim = word_embedding_dim * max_sentence_length
word_embeddings_source = nn.Embedding(n_words_source, word_embedding_dim, max_norm=1).data.zero_()
word_embeddings_target = nn.Embedding(n_words_target, word_embedding_dim, max_norm=1).data.zero_()

#A_source = np.diag(np.ones(nw_source))
#A_target = np.diag(np.ones(nw_target))
for row in source_sentences:
    for i in range(0,max_sentence_length):
#        A_source[W2I_source[row[i]],W2I_source[row[i]]] += 1.0
#        A_source[W2I_source[row[i+1]],W2I_source[row[i]]] += 1.0
        if i < len(row)+2:
            if i == len(row) -1:
                word_embeddings_source[w2i_s[row[i]],w2i_s['']] += 1.0
            elif i > len(row) -1:
                word_embeddings_source[w2i_s[''],w2i_s['']] += 1.0
            else:
                word_embeddings_source[w2i_s[row[i]],w2i_s[row[i+1]]] += 1.0

decay_rate = 1.0

for i in range(A_source.shape[0]):
    A_source[i,:] = A_source[i,:]/np.linalg.norm(A_source[i,:])
    
def series(A,n=2,beta=0.5):
    A += beta * A.dot(A)
    return A

#A_source = series(A_source)
A_source = expm(decay_rate*A_source)
np.fill_diagonal(A_source,0)
#A_source=np.triu(A_source,1)
#A_source = sigmoid(A_source)
#A_source = np.linalg.pinv(np.diag(np.ones(nw_source))-decay_rate*A_source)

for row in Target:
    for i in range(0,maxL_target):
#        A_target[W2I_target[row[i]],W2I_target[row[i]]] += 1.0
#        A_target[W2I_target[row[i+1]],W2I_target[row[i]]] += 1.0
        if i < len(row)+1:
            if i == len(row) -1:
                A_target[W2I_target[row[i]],W2I_target['']] += 1.0
            elif i > len(row) -1:
                A_target[W2I_target[''],W2I_target['']] += 1.0
            else:
                A_target[W2I_target[row[i]],W2I_target[row[i+1]]] += 1.0
                
for i in range(A_target.shape[0]):
    A_target[i,:] = A_target[i,:]/np.linalg.norm(A_target[i,:])
    
#A_target = series(A_target)
  
A_target = expm(decay_rate*A_target)
np.fill_diagonal(A_target,0)
#A_target=np.triu(A_target,1)
#A_target = sigmoid(A_target) 
#A_target = np.linalg.pinv(np.diag(np.ones(nw_target))-decay_rate*A_target)

In [4]:
# 词向量表示
word_embedding_dim = 15
sentence_embedding_dim = word_embedding_dim * max_sentence_length
word_embeddings_source = nn.Embedding(n_words_source, word_embedding_dim, max_norm=1)
word_embeddings_target = nn.Embedding(n_words_target, word_embedding_dim, max_norm=1)

In [5]:
# 定义一个深层神经网络
class DNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_hidden=1, activation=None, out_activation=None, enable_dropout=False):
        super(DNN,self).__init__()
        self.input = nn.Linear(input_size, hidden_size)
        hiddens = []
        self.n_hidden = n_hidden
        self.activation = activation
        self.enable_dropout = enable_dropout
        self.dropout = nn.Dropout(p=0.05)
        for n in range(self.n_hidden):
            hiddens.append(nn.Linear(hidden_size, hidden_size))
        self.hiddens = nn.Sequential(*hiddens)
        self.output = nn.Linear(hidden_size, output_size)
        self.out_activation = out_activation
    def forward(self, x):
        x = self.input(x)
        if self.activation is not None:
            x = self.activation(x)
        for i in range(self.n_hidden):
            x = self.hiddens[i](x)
            if self.enable_dropout:
                self.dropout(x)
            if self.activation is not None:
                x = self.activation(x)
        x = self.output(x)
        if self.out_activation is not None:
            x = self.out_activation(x).clamp(1e-8,1-1e-7)
        return x

# LSTM
class LSTM(nn.Module):
    def __init__(self, seq_size, batch_size, input_size, hidden_size, n_hidden=1, in_layers=None, out_layers=None):
        super(LSTM,self).__init__()
        self.in_layers = in_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_hidden = n_hidden
        self.seq_size = seq_size
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size, self.hidden_size, num_layers=n_hidden)
        self.out_layers = out_layers
#        self.init_hidden()
    
#    def init_hidden(self):
#        self.hx =(torch.randn(self.n_hidden, self.batch_size, self.hidden_size), torch.randn(self.n_hidden, self.batch_size, self.hidden_size))

    def forward(self, x):
        self.batch_size = x.size(0)
#        self.hx=self.init_hidden()
        if self.in_layers is not None:
            x = self.in_layers(x)
            x = x.contiguous().view(self.batch_size, self.seq_size, self.input_size)
        x = torch.transpose(x,0,1)
        x, hx = self.lstm(x)
        x = torch.transpose(x,0,1)
        if self.out_layers is not None:
            x = x.contiguous().view(self.batch_size,-1)
            x = self.out_layers(x)
        return x
    
    
# 定义一个函数，将输入句子文字转化为矢量编码
def get_sentence_embedding(S,mode='source'):
    if mode == 'source':
        idx = [w2i_s[w] for w in S]
        for i in range(len(S), max_sentence_length):
            idx.append(0)
        idx = torch.LongTensor([idx])
        return word_embeddings_source(idx)#.view(1,-1)
    elif mode == 'target':
        idx = [w2i_t[w] for w in S]
        for i in range(len(S), max_sentence_length):
            idx.append(0)
        idx = torch.LongTensor([idx])
        return word_embeddings_target(idx)#.view(1,-1)
    else:
        print('mode error.')
        return None

    
def target_sentence_embedding2words(embs):
    embs = embs.view(max_sentence_length,-1)
    words = []
    for i in range(max_sentence_length):
        loss = torch.mean((word_embeddings_target.weight.detach()-embs[i].detach())**2,1)
        words.append(i2w_t[torch.argmin(loss).tolist()])
    return u' '.join(words)

def source_sentence_embedding2words(embs):
    embs = embs.view(max_sentence_length,-1)
    words = []
    for i in range(max_sentence_length):
        loss = torch.mean((word_embeddings_source.weight.detach()-embs[i].detach())**2,1)
        words.append(i2w_s[torch.argmin(loss).tolist()])
    return u' '.join(words)

def get_loser(loss_g=None,loss_d=None):
    if loss_g > loss_d:
        return 'G'
    elif loss_g < loss_d:
        return 'D'
    else:
        if np.random.random()<0.5:
            return 'G'
        else:
            return 'D'

In [8]:
# 模型参数设置与模型


dnn_input_size = max_sentence_length* word_embedding_dim
dnn_output_size = 1
dnn_hidden_size = 64
dnn_activation = torch.relu

batch_size = 5


gnn_input_size = 20
gnn_output_size = max_sentence_length * word_embedding_dim
gnn_hidden_size = 64
gnn_activation = torch.relu

# 定义一个 Generator, 一个 Discriminator

#G = DNN(g_input_size, g_output_size, g_hidden_size, n_hidden=n_hidden, activation=g_activation)
#D = DNN(d_input_size, d_output_size, d_hidden_size, n_hidden=n_hidden, activation=d_activation, out_activation=torch.sigmoid)
g_in_layers = DNN(gnn_input_size, gnn_output_size, gnn_hidden_size, n_hidden=2, activation=gnn_activation, out_activation=None)
d_out_layers = DNN(dnn_input_size, dnn_output_size, dnn_hidden_size, n_hidden=2, activation=dnn_activation, out_activation=torch.sigmoid)

G = LSTM(max_sentence_length, batch_size, word_embedding_dim, word_embedding_dim, n_hidden=1, in_layers=g_in_layers)
D = LSTM(max_sentence_length, batch_size, word_embedding_dim, word_embedding_dim, n_hidden=1, out_layers=d_out_layers)
DLoss = []
GLoss = []

In [9]:
# 模拟参数设置

lr = 1e-3

data_size = len(target_sentences)
n_epochs = 50000

d_steps = 1
g_steps = 1


torch.manual_seed(1)
np.random.seed(1)

g_optimizer = torch.optim.Adam(G.parameters(),lr=lr)
d_optimizer = torch.optim.Adam(D.parameters(),lr=lr)

def lr_lambda(epoch):
    return 0.99**(epoch/50.0)

g_scheduler = torch.optim.lr_scheduler.LambdaLR(g_optimizer, lr_lambda, last_epoch=-1)
d_scheduler = torch.optim.lr_scheduler.LambdaLR(d_optimizer, lr_lambda, last_epoch=-1)

criterion = nn.BCELoss() #torch.nn.SmoothL1Loss() # or torch.nn.MSELoss()

display_interval = 100

# 模型优化
#loser = 'D'
for epoch in range(n_epochs):
    
    d_loss = []
    g_loss = []
    
    for d in range(d_steps):
        
        # 归零D的梯度
        G.zero_grad()
        D.zero_grad()
        
        # 生成一个 batch 的真实样本
        indices = np.random.randint(0, data_size, batch_size)
        real_samples = get_sentence_embedding(target_sentences[indices[0]], mode='target')
        for i in range(1, batch_size):
            real_samples = torch.cat((real_samples, get_sentence_embedding(target_sentences[indices[i]], mode='target')))
        real_samples = real_samples.detach()
        d_real_err = criterion(D(real_samples),torch.ones(batch_size, 1))
        d_loss.append(d_real_err.tolist())
        
        d_real_err.backward()
        d_optimizer.step()

        # 归零D的梯度
        G.zero_grad()
        D.zero_grad()
        
        # 生成一个 batch 的虚假样本
        if np.random.random()<0.9:
            fake_samples = G(torch.rand(batch_size, gnn_input_size))
        else:
            fake_samples = torch.randn(batch_size, max_sentence_length, word_embedding_dim)
        
        d_fake_err = criterion(D(fake_samples),torch.zeros(batch_size, 1))
        d_loss.append(d_fake_err.tolist())
        
        d_fake_err.backward()
        d_optimizer.step()
        d_scheduler.step()
        
    for g in range(g_steps):
        
        G.zero_grad()
        D.zero_grad()
        if np.random.random()<0.9:
            fake_samples = G(torch.rand(batch_size, gnn_input_size))
            g_err = criterion(D(fake_samples),torch.ones(batch_size, 1))
        else:
            fake_samples = G(-torch.rand(batch_size, gnn_input_size))
            g_err = criterion(D(fake_samples),torch.zeros(batch_size, 1))
        g_loss.append(g_err.tolist())
        
        g_err.backward()
        g_optimizer.step()
        g_scheduler.step()
        
    if len(d_loss)>0:
        DLoss.append(np.mean(d_loss))
    elif len(DLoss)>0:
        DLoss.append(DLoss[-1])
    else:
        DLoss.append(np.inf)
    if len(g_loss)>0:
        GLoss.append(np.mean(g_loss))
    elif len(GLoss)>0:
        GLoss.append(GLoss[-1])
    else:
        GLoss.append(np.inf)
        
    if epoch % display_interval == 0:
        print(r'epoch {}, G-loss {}, D-loss {}'.format(epoch, GLoss[-1], DLoss[-1]))
        print(target_sentence_embedding2words(G(torch.randn(1, gnn_input_size)).view(-1).detach()))

epoch 0, G-loss 0.761330604553, D-loss 0.699897348881
Thursday Thursday Luis Thursday Thursday Thursday Thursday Thursday Thursday Thursday Thursday Luis Thursday Thursday Thursday Luis Luis Thursday Luis Luis Thursday Thursday Thursday Thursday Thursday Thursday Thursday Thursday Thursday Luis
epoch 100, G-loss 2.7599811554, D-loss 0.436667751521
Navarro days single single single single single single single single single single single single single single single single single single January Padilla luggage luggage luggage luggage    
epoch 200, G-loss 5.34423017502, D-loss 0.00539134198334
Padilla January days days days Lidia days Escrig Lidia Amelia Amelia Ram'irez Alicia Padilla Alicia Ram'irez suitcase Jaime too        Ballester  Ballester Padilla
epoch 300, G-loss 5.01646900375e-05, D-loss 0.0532207291108
the Montero Torres twenty-ninth Lidia Montero Lidia you Lidia Ramos number Padilla Padilla Padilla leave Redondo Botella Revilla send Izquierdo Concepci'on L'opez Varela call 'An

epoch 3500, G-loss 9.13814067841, D-loss 8.65606452862e-05
Thursday Torres days shower Montoro Montoro Montoro Lidia checks am Padilla Padilla Padilla Padilla Padilla Padilla any twenty-fourth  too   cold    give Padilla Padilla luggage
epoch 3600, G-loss 8.8805770874, D-loss 8.13872557046e-05
would twenty-second sixth Fern'andez Padilla shower room Botella Rosa reservation Luis ninth days Ana V'azquez number form Cabo  extras    Jaime Balaguer  Ballester Pedro putting 
epoch 3700, G-loss 9.73970413208, D-loss 1.94967224236e-05
would Rubio sixth Fern'andez Cerezo shower room Botella Mrs reservation shower  Montoro Ana V'azquez number Concepci'on Cabo a extras   week Jaime Balaguer  Ballester Pedro Peinado 
epoch 3800, G-loss 9.98909759521, D-loss 3.69602312276e-05
would Rubio sixth Fern'andez Padilla shower room Botella Mrs reservation shower  Montoro Ana V'azquez number Concepci'on Cabo a extras   week luggage Balaguer  Ballester Pedro Peinado 
epoch 3900, G-loss 9.70452785492, D-loss

KeyboardInterrupt: 

In [ ]:
window_size = 50
avg_losses = np.array(DLoss)[:len(DLoss)//50 *50].reshape([-1,window_size]).mean(1)
pl.plot(np.arange(0,len(DLoss)//50 *50,window_size), avg_losses,'r-',label='D-loss')
avg_losses = np.array(GLoss)[:len(GLoss)//50 *50].reshape([-1,window_size]).mean(1)
pl.plot(np.arange(0,len(GLoss)//50 *50,window_size), avg_losses,'g--',label='G-loss')
pl.legend(loc='best')
pl.xlabel('Time')
pl.ylabel('Loss')
pl.yscale('log')

In [ ]:
real_samples.size(0)

In [ ]:
word_embeddings_target.weight[:10]

In [ ]:
real_samples.tolist()

In [ ]:
torch.transpose()

In [ ]:
criterion(torch.zeros(5,1).clamp(1e-8,1-1e-7),torch.zeros(5,1))

In [ ]:
pl.imshow(D.parameters().next().detach().numpy())

In [ ]:
D(fake_samples)

In [ ]:
np.mean([])

In [ ]:
np.nan<np.nan

In [ ]:
word_embeddings_target.weight[0]